<a href="https://colab.research.google.com/github/sharvaniyadav/Automated-Component-Detection-in-PCB-Manufacturing-Using-YOLOv8-Project-3/blob/main/Automated_Component_Detection_in_PCB_Manufacturing_Using_YOLOv8_Project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>